In [5]:
import numpy as np
import pandas as pd
import ee
import geemap
from csv import reader
import scipy.interpolate

ee.Initialize()

In [10]:
"""
csv header legend:
-0:  Mininum Longitude
-1:  Minimum Latitude
-2:  Maximum Longitude
-3:  Maximum Latitude
-4:  Percent Vegetation Loss
-5:  Percent Bare Initial
-6:  Percent Significant VH Values
-7:  Average NIR/G
-8:  Average SWIR1/B
-9:  NASA Elev
-10: GEDI Elev
-11: Elev Loss
"""

## Method 1: Scoring

In [11]:
# read the csv into an ndarray
with open('results/compiled_passing.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader, None) # skip header
    results = np.array([[float(e) for e in row[:-1]] for row in csv_reader]) # cut off last col (pass/fail)

In [ ]:
# create a list of center coordinates for each pixel
min_lat = results[:,0]
min_lon = results[:,1]
max_lat = results[:,2]
max_lon = results[:,3]

mid_lat = [(l1 + l2) / 2 for l1, l2 in zip(min_lat, max_lat)]
mid_lon = [(l1 + l2) / 2 for l1, l2 in zip(min_lon, max_lon)]
xy_pairs = list(zip(mid_lat, mid_lon)) 

In [ ]:
# create an elevation interpolator using center coordinates and NASADEM values
nasa_elev = results[:,9]
interp1 = scipy.interpolate.LinearNDInterpolator(xy_pairs, nasa_elev)

#
pixel_size_m = 250
dist = pixel_size_m * 0.00001 # roughly 250m

# use the interpolator to calculate scores
scores = []
for i, (lat, lon) in enumerate(xy_pairs):
    # elevation of the center pixel
    center = interp1(lat, lon)
    
    # grab the elevation for the 8 surrounding pixels using the interpolate function
    left = lat + dist
    right = lat - dist
    top = lon + dist
    bot = lon - dist

    surrounding = [interp1(left, lon),  # left
                   interp1(left, top),  # top left corner
                   interp1(lat, top),   # top
                   interp1(right, top), # top right corner
                   interp1(right, lon), # right
                   interp1(right, bot), # bottom right corner
                   interp1(lat, bot),   # bottom
                   interp1(left, bot)]  # bottom left corner
    
    # count how many surrounding pixels are above the center pixel
    score = 0 
    for height in surrounding:
        if center < height: 
            score += 1
    scores.append(score)
    
# print(len(mid_lat))
# print(len(scores))
# print(scores)

In [ ]:
# write the scores to the csv
df = pd.read_csv("results/compiled_passing.csv")
df["Score"] = scores
df.to_csv("results/compiled_passing_with_score.csv", index=False)

In [ ]:
# # Wrap geometry list in a Feature Collection
# fc = ee.FeatureCollection(passed)

# # Export the Feature Collection to Google Earth Engine (GEE)
# task = ee.batch.Export.table.toAsset(**{
#   'collection': fc,
#   'description':'compiled_results',
#   'assetId': 'users/raymondeah/compiledResultsInterpRoi5', # change to your GEE Asset path and a unique name (will not overwrite already existing assets, so old names cannot be reused)
# })

# task.start()

## Method 2: Crosshairs

In [ ]:
# read the csv into an ndarray
with open('results/compiled_passing.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader, None) # skip header
    results = np.array([[float(e) for e in row[:-1]] for row in csv_reader]) # cut off last col (pass/fail)

In [ ]:
# create a list of center coordinates for each pixel
min_lat = results[:,0]
min_lon = results[:,1]
max_lat = results[:,2]
max_lon = results[:,3]

mid_lat = [(l1 + l2) / 2 for l1, l2 in zip(min_lat, max_lat)]
mid_lon = [(l1 + l2) / 2 for l1, l2 in zip(min_lon, max_lon)]
xy_pairs = list(zip(mid_lat, mid_lon)) 

In [8]:
"""
Given a center point, returns specified amount of layers of sample points
in 8 directions (N, E, S, W, NE, NW, SE, SW)

:param center: tuple containing lat/lon coordinates of center point
:param size:   how many layers to create
:param dist:   distance in meters between each layer

:return:       a list of all layers
"""
def create_layers(center, size, dist, interp):
    layers = []
    lat, lon = center # unpack tuple
    dist = dist * 0.00001 # meters to degrees
    center_elev = interp(lat, lon) # elevation of the center pixel
        
    for i in range(1, size+1):
        # grab the elevation for the 8 surrounding pixels using the interpolate function
        left = lat + (dist * i)
        right = lat - (dist * i)
        top = lon + (dist * i)
        bot = lon - (dist * i)
        
        surrounding = [interp(lat, top),   # N
                       interp(right, top), # NE
                       interp(right, lon), # E
                       interp(right, bot), # SE
                       interp(lat, bot),   # S
                       interp(left, bot),  # SW
                       interp(left, lon),  # W
                       interp(left, top)]  # NW
                       
        layers.append(surrounding)
        
    return layers

In [12]:
CROSSHAIR_SIZE = 3 # set how many pixel layers to extend the 'crosshair'
DIST = 250 # meters

# create an elevation interpolator using center coordinates and NASADEM values
nasa_elev = results[:,9]
interp1 = scipy.interpolate.LinearNDInterpolator(xy_pairs, nasa_elev)

# generate scores based on mean layer elevation compared to center
scores = []
for lat, lon in xy_pairs:
    # elevation of the center point
    center_elev = interp1(lat, lon)
    
    # create the layers
    layers = create_layers((lat, lon), CROSSHAIR_SIZE, DIST, interp1)
    
    # calculate a score: take the mean elevation of the layers, how many are above the center?
    score = 0
    for layer in layers:
        mean = sum(layer) / len(layer)
        if mean > center_elev:
            score += 1
    scores.append(score)
    
# print(len(mid_lat), '=?', len(scores))
# print(scores)

In [ ]:
# write the scores to the csv
df = pd.read_csv("results/compiled_passing.csv")
df["Score"] = scores
df.to_csv("results/compiled_passing_with_score.csv", index=False)